# GRAIN G600:
Modelos de clasificación de trigo y maiz, se entrena un modelo para clasificar todo el grano.

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import  imblearn
from imblearn.over_sampling import  ADASYN
from tensorboard.plugins.hparams import api as hp


import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime
#from PIL import image
#import tensorflow-gpu
#from bayes_opt import BayesianOptimization


print(tf.__version__)
#tf.debugging.set_log_device_placement(True)
#-------------------------------------------------------------------------------
import time





Iniciamos configurando tensorflow para usar la GPU, al igual que tensorboard

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if len(tf.config.list_physical_devices('GPU'))>0:
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    print("Virtual GPU devices: ", tf.config.list_logical_devices('GPU'))

In [ ]:
tf.debugging.set_log_device_placement(False)

Metricas a seguir:

In [ ]:

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'
METRIC_LOSS = 'loss'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'), hp.Metric(METRIC_LOSS, display_name='Loss')],
  )

## Ambos granos: Trigo y Maiz
Generamos dos iteradores de carpetas para los conjuntos de datos de validación y de entrenamiento.

In [ ]:
# Directorios de entrenamiento y validación r"../../grainspace/M600/pre/both/
train_dir = '../grainspace/M600/pre/both/train'
validation_dir = '../grainspace/M600/pre/both/validation'

datagen = ImageDataGenerator(
    rescale=1./255,          # Normaliza los valores de los píxeles
    shear_range=0.2,         # Aplica transformaciones de corte
    zoom_range=0.2,          # Aplica zoom aleatorio
    horizontal_flip=True     # Invierte las imágenes horizontalmente
)
train_generator = datagen.flow_from_directory(
    train_dir,  # Directorio con las imágenes de entrenamiento
    target_size=(150, 150),     # Tamaño de las imágenes
    batch_size=32,              # Tamaño del lote
    class_mode='categorical'         # Tipo de clasificación (binary, categorical, etc.)
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)







In [ ]:
batch_images, batch_labels = next(train_generator)

# Assuming the images are in the first channel of the batch
sample_images = batch_images[:, :, :, 0]

# Plot the first few images
num_images_to_plot = 5
fig, axes = plt.subplots(1, num_images_to_plot, figsize=(15, 5))
for i in range(num_images_to_plot):
    axes[i].imshow(sample_images[i])
    axes[i].axis('on')
plt.show()

Definimos la arquitectura del modelo, de forma iterativa, para obetner los mejores hiperparametros.

In [ ]:
print(train_generator.class_indices)

In [ ]:
def train_test_model(hparams,logdir):

  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(hparams[HP_NUM_UNITS], (3, 3), activation='relu'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation=tf.nn.softmax),
  ])
  tensorboard = TensorBoard(log_dir="logs/G600_both/HPtuning/{}".format(time.time()))
  early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='binary_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(train_generator,
            steps_per_epoch=train_generator.samples // train_generator.batch_size,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // validation_generator.batch_size,
            epochs=1,
            callbacks=[tensorboard,early_stopping, # log metrics
                      hp.KerasCallback(logdir, hparams)])  # log hparams
                       # Run with 1 epoch to speed things up for demo purposes
  loss, accuracy = model.evaluate(validation_generator)
  model.summary()
  print('Loss: {0:.4f}, Accuracy: {1:.4f}'.format(loss, accuracy))
  return loss, accuracy
            

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    loss, accuracy = train_test_model(hparams,run_dir)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
"""model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=90,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ]
)"""

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

In [8]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message
    }
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    return response

# Reemplaza con tu token y chat_id
telegram_token = "7448990665:AAEBnXWdATGjB6UOQRdYW6uztDbzjDk8OA4"
chat_id = 1298690395
message = "¡El codigo en VS Code ha terminado!"

# Envía la notificación
response = send_telegram_message(telegram_token, chat_id, message)
print(response.json())


{'ok': True, 'result': {'message_id': 4, 'from': {'id': 7448990665, 'is_bot': True, 'first_name': 'VS Code Done!', 'username': 'VSC_noti_bot'}, 'chat': {'id': 1298690395, 'first_name': 'Omarciano', 'type': 'private'}, 'date': 1723268183, 'text': '¡La compilación en VS Code ha terminado!'}}
